### Capstone GDP & Suicide: A Global Correlation

Beginning to import code & conduct preliminary analysis

In [14]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# load data from excel
df = pd.read_excel('C:/Users/jlevi/code/savvycoders/Capstone/suicide capstone/FinalDatasets/TransposedDataSet.xlsx')

# preliminary data check
#print(df.head())
#print(df.info())
#print(df.describe())

# Finish Cleaning Data:
# remove country Turkey b/c missing a lot of values 
df = df[df['Country'] != 'Turkiye']

# filter to years 1995-2017
df = df[(df['Year'] >= 1995) & (df['Year'] <= 2017)]

# check
print(df['Country'].value_counts())

Australia          23
Austria            23
Portugal           23
Slovak Republic    23
Spain              23
Sweden             23
Switzerland        23
United Kingdom     23
United States      23
Brazil             23
Chile              23
Estonia            23
Israel             23
Romania            23
Russia             23
Slovenia           23
South Africa       23
Colombia           23
Latvia             23
Lithuania          23
Costa Rica         23
Poland             23
Norway             23
New Zealand        23
Greece             23
Belgium            23
Bulgaria           23
Canada             23
Czech Republic     23
Denmark            23
Finland            23
France             23
Germany            23
Croatia            23
Netherlands        23
Hungary            23
Iceland            23
Ireland            23
Italy              23
Japan              23
Korea              23
Luxembourg         23
Mexico             23
Argentina          23
Name: Country, dtype: int64


### Preliminary Analysis

**Steps:**
1. Calculate descriptive statistics for my var. of interest to gain understanding of distribution & variation of data
2. Create scatter plots to visualize relationship between GDP & suicide rates for each country & year. **Check for outliers** or patterns
3. Compute correlation coefficients to measure strength & direction of the linear relationship between GDP & suicide rates for each country & year. **Check for significant correlations**
4. Run regression models to examine relationship
* Unable to control for other variables such as population, income, inequality, & mental health expenditure, that may affect the relationship due to time/data constraints 
* Panel data - effect of GDP on suicide rate within each country & want to control for country specific factors that are constant over time (cultural stigma w mental health, cultural attitude to drinking, etc)
* Time series data - overall effect of GDP on suicide rate across all countries over time
5. Perform hypothesis tests to determine if relationship is statistically significant 

### Later part 

In [ ]:
# run linear regression model
X = df[['GDP_USD']] # independent variable
y = df['Suicide_Rate'] # dependent variable
model = LinearRegression().fir(X,y)

# print the model coefficients
#print(model.intercept_)
#print(model.coef_)



In [ ]:
# use statsmodels to run regression & print summary
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())